# C++ Test

## $\bar y(t)$

In [1]:
#include <functional>
#include <iostream>
#include <vector>

class Subfunction {
public:
    Subfunction(std::function<double(double)> func) : m_func(func) {}

    double operator()(double t) const {
        return m_func(t);
    }
    Subfunction operator+(const Subfunction& other) const {
        return Subfunction([=](double t) { return m_func(t) + other.m_func(t); });
    }
    Subfunction operator-(const Subfunction& other) const {
        return Subfunction([=](double t) { return m_func(t) - other.m_func(t); });
    }
    Subfunction operator*(const Subfunction& other) const {
        return Subfunction([=](double t) { return m_func(t) * other.m_func(t); });
    }
    Subfunction operator/(const Subfunction& other) const {
        return Subfunction([=](double t) { return m_func(t) / other.m_func(t); });
    }

private:
    std::function<double(double)> m_func;
};


Subfunction f([](double t) { return t * t; });
Subfunction g([](double t) { return t + 2; });

std::cout << (f + g)(3);

14

In [2]:
class PiecewiseFunction {
public:
    PiecewiseFunction(const std::vector<double>& times, const std::vector<Subfunction>& functions)
        : m_times(times), m_functions(functions) {}

    double operator()(double t) const {
        for (size_t i = 0; i < m_times.size(); ++i) {
            if (t < m_times[i]) {
                return m_functions[i](t);
            }
        }
        return m_functions.back()(t);
    }
        
    PiecewiseFunction operator+(const PiecewiseFunction& other) const {
        std::vector<Subfunction> new_functions;
        for (size_t i = 0; i < m_functions.size(); ++i) {
            new_functions.push_back(m_functions[i] + other.m_functions[i]);
        }
        return PiecewiseFunction(m_times, new_functions);
    }

    PiecewiseFunction operator-(const PiecewiseFunction& other) const {
        std::vector<Subfunction> new_functions;
        for (size_t i = 0; i < m_functions.size(); ++i) {
            new_functions.push_back(m_functions[i] - other.m_functions[i]);
        }
        return PiecewiseFunction(m_times, new_functions);
    }

    PiecewiseFunction operator*(const PiecewiseFunction& other) const {
        std::vector<Subfunction> new_functions;
        for (size_t i = 0; i < m_functions.size(); ++i) {
            new_functions.push_back(m_functions[i] * other.m_functions[i]);
        }
        return PiecewiseFunction(m_times, new_functions);
    }

    PiecewiseFunction operator/(const PiecewiseFunction& other) const {
        std::vector<Subfunction> new_functions;
        for (size_t i = 0; i < m_functions.size(); ++i) {
            new_functions.push_back(m_functions[i] / other.m_functions[i]);
        }
        return PiecewiseFunction(m_times, new_functions);
    }
private:
    std::vector<double> m_times;
    std::vector<Subfunction> m_functions;
};

// Define some Subfunctions
Subfunction f1([](double t) { return t * t; });
Subfunction f2([](double t) { return 2 * t + 1; });
Subfunction f3([](double t) { return t - 3; });

// Define time intervals (without initial time 0)
std::vector<double> times = {1, 2, 3};

// Define PiecewiseFunctions
PiecewiseFunction pwf1(times, {f1, f2, f3});
PiecewiseFunction pwf2(times, {f2, f3, f1});

// Evaluate PiecewiseFunctions at 1.5
(pwf1 + pwf2)(1.5)

2.5000000

In [ ]:
#include <iostream>
#include <vector>
#include <functional>
#include <numeric>
#include <stdexcept>
#include <algorithm>

// Assuming Subfunction and PiecewiseFunction classes are defined as provided earlier


// Define some Subfunctions (piecewise constant)
Subfunction g1([](double t) { return 4.0; });
Subfunction g2([](double t) { return 5.0; });
Subfunction g3([](double t) { return 7.0; });

// Define time intervals (without initial time 0)
std::vector<double> times = {2, 4, 8};

// Define PiecewiseFunction for g
PiecewiseFunction g(times, {g1, g2, g3});

// Compute the integral of the piecewise constant function g
std::vector<Subfunction> integral_functions;

for (size_t i = 0; i < times.size(); ++i) {
    Subfunction integral_func([=](double t) {
        size_t q_t = std::upper_bound(times.begin(), times.end(), t) - times.begin() + 1;
        double integral_value = 0.0;

        // Add the sum of the intervals [T_j, T_{j+1}] for j from 0 to q(t)-1
        for (size_t j = 0; j <= q_t - 1; ++j) {
            integral_value += g(j==0 ? 0 : times[j-1]) * (times[j] - (j==0 ? 0 : times[j-1]));
        }
        // Add the remaining term -g_{q(t)-1} * (T_{q(t)} - t)
        integral_value -= g((q_t > 1) ? times[q_t - 2] : 0) * (times[q_t - 1] - t);
        
        return integral_value;
    });
    
    integral_functions.push_back(integral_func);
}

PiecewiseFunction g_integral(times, integral_functions);

// Evaluate the integral at different times

(std::vector<double>){g_integral(0.5), g_integral(2.5), g_integral(5)}

## valarray

valarray 可以直接丟整進 cmath function 裡

In [2]:
#include <valarray>
#include <iostream>
#include <cmath>

std::valarray<double> nums = {1, 2, 3, 4, 5};
std::valarray<double> sqrts = sqrt(nums);

for(const double& num : sqrts)
    std::cout << num << ", "; 


1, 1.41421, 1.73205, 2, 2.23607, 

## Range Enumeration 貌似不能用 Temporary Object

下面這樣不行：

In [ ]:
#include <valarray>
#include <iostream>
#include <cmath>

std::valarray<double> nums = {1, 2, 3, 4, 5};

for(const double& num : sqrt(nums))
    std::cout << num << ", "; 


cast 成 `valarray<double>` 就可以：

In [4]:
#include <valarray>
#include <iostream>
#include <cmath>

std::valarray<double> nums = {1, 2, 3, 4, 5};

for(const double& num : static_cast<std::valarray<double>>(sqrt(nums)))
    std::cout << num << ", "; 


1, 1.41421, 1.73205, 2, 2.23607, 

## [CppQuiz.org](https://cppquiz.org/quiz/question/225)

## py

* structured binding
* `std::max`（定義在 algorithm 裡）對三個數以上要這樣用 `std::max({1, 2, 3})`。直接寫 `std::max(1, 2, 3)` 的時候第三個 arg 是 comp
* python list append = c++ vector push_back
* python `math.inf` = c++ `std::numeric_limits<double>::infinity()`，定義在 `<limit>`
* `deque` 比較

## `deque<T>` 的 `pop_front()` 和 `pop_back()` 不回傳 element

* [如果設計成回傳 element 會有兩個問題：](https://stackoverflow.com/questions/25035691/why-doesnt-stdqueuepop-return-value)
    * return element 會呼叫 copy ctor。如果 copy ctor 在 return 時 throw exception，該 element 也已經消失了
    * 對於不真正需要 popped element 的程序，return 時的 copy 是浪費時間

In [ ]:
template<class T>
class queue {
    T* elements;
    std::size_t top_position;
    // stuff here
    T pop()
    {
        auto x = elements[top_position];
        // TODO: call destructor for elements[top_position] here
        --top_position;  // alter queue state here
        return x;        // calls T(const T&) which may throw
    }
};

## Ranges (C++ 20)

* Functional programming in C++
* [Cppcon presentation](https://youtu.be/FRkJCvHWdwQ?t=878)

In [ ]:
vector data {1, 2, 3, 4, 5, 6, 7, 8, 9, 10};
auto result { data
    | views::filter([](const auto& value) { return value % 2 == 0; })/* 2 4 6 8 10*/
    | views::transform([](const auto& value) { return value * 2.0; })/* 4 8 12 16 20 */
    | views::drop(2)                                                 /* 12 16 20 */
    | views::reverse                                                 /* 20 16 12 */
    | views::transform([](int i) { return to_string(i); })           /* "20" "16" "12" */
}

## Comparator

[這裡](http://fusharblog.com/3-ways-to-define-comparison-functions-in-cpp/)和[這裡](https://docs.oracle.com/cd/E19205-01/819-3701/com_5335.htm)都說 ```>=``` 不能當 comparator 因為會破壞 ```If f(x, y) then !f(y, x)``` （antisymmetry）。但測試起來沒問題啊？？？

In [1]:
#include <algorithm>
#include <vector>

std::vector<int> a = {1, 2, 3, 3, 3, 4, 5, 6, 7, 8, 8, 9};

std::sort(a.begin(), a.end(), [](int x, int y){ return x >=y; }); 

a

{ 9, 8, 8, 7, 6, 5, 4, 3, 3, 3, 2, 1 }

## Raw String

In [1]:
#include <iostream>

std::cout << R"(This is a \n "raw string")"<< std::endl;

This is a \n "raw string"


## ```std::greater``` and ```std::sort```

* [3 Ways to Define Comparison Functions in C++](http://fusharblog.com/3-ways-to-define-comparison-functions-in-cpp/)

In [6]:
#include <iostream>     // std::cout
#include <functional>   // std::greater
#include <algorithm>    // std::sort
#include <vector>

int main () {
    std::vector<int> a = {1, 2, 3, 4, 5, 6, 7, 8, 9};
    
    std::sort(a.begin(), a.end(), std::greater<int>());
    
    for (const int& value : a)
        std::cout << value << ", ";
    
    return 0;
}

main();

9, 8, 7, 6, 5, 4, 3, 2, 1, 

## ```std::to_string``` and ```std::stoi```

In [4]:
#include <iostream>
#include <string>  // std::to_string and std::stoi
#include <vector>

int a = 123;

std::vector<int> v = {};
for(char& c : std::to_string(a))
    v.push_back(std::stoi(std::string(1, c)));

v

{ 1, 2, 3 }

## Returning a Tuple

In [2]:
#include <tuple>
#include <iostream>

std::tuple<int, int> f(){   // cannot define as auto
    return {1, 2};
}

std::cout << std::get<0>(f()) << ", " << std::get<1>(f()) << std::endl;

1, 2


## Loop Over a Tuple? 

In [2]:
#include <iostream>

for (auto x : {1, 2, 3, 4, 5})
    std::cout << x << ", ";

1, 2, 3, 4, 5, 

## String Literal

In [5]:
#include <iostream>
#include <string>

{
//     const char* s = "aaa";  // 可以宣告但不能改內容。string literal 是存在 const 區
//     std::string s = "aaa";  // 可以宣告也可以改內容
//     char* s = "aaa";        // interpreter 不給過。因為 string literal 是 const 所以這樣宣告不合法
    char s[] = "aaa";       // 宣告成 array：可以宣告也可以改內容。其實這樣宣告
    
    s[0] = 'b';
    std::cout << s << std::endl;
}

baa
